# Setup

## Importações

In [1]:
import requests as req
import pandas as pd
import json
from datetime import datetime
from google.colab import userdata

## Constantes

In [2]:
DOMAIN = 'https://api.github.com/graphql'
TOKEN = userdata.get('token')
HEADERS = {
  'Authorization': f'bearer {TOKEN}',
  'Content-Type': 'application/json'
}
TODAY = datetime.now()
NUMBER_OF_REPOSITORIES=30

## Query Geral

In [3]:
query_geral = """
query geral_query {{
  search(query: "stars:>{0}", type: REPOSITORY, first: {0}) {{
    nodes {{
      ... on Repository {{
        nameWithOwner
        updatedAt
        createdAt
        releases{{
          totalCount
        }}
        nameWithOwner
        issues{{
          totalCount
        }}
        closedIssues: issues(states: CLOSED) {{
          totalCount
        }}
         primaryLanguage {{
          name
        }}
        releases{{
          totalCount
        }}
        pullRequests(states:MERGED){{
          totalCount
        }}
      }}
    }}
  }}
}}
""".format(NUMBER_OF_REPOSITORIES)

## Funções

In [4]:
def doPost(data : json)-> json:
  response = req.post(DOMAIN, headers=HEADERS, json=data)
  if response.status_code == 200:
    return response.json()

  raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}')



def analisar_contribuicoes(repositorios):
    repo_data = []
    for repo in repositorios.get('data', {}).get('search', {}).get('nodes', []):
        if isinstance(repo, dict):
            repo_name = repo.get('nameWithOwner', '')
            pull_requests_data = repo.get('pullRequests', {})
            total_pull_requests = pull_requests_data.get('totalCount', 0)
            repo_data.append({'Repositório': repo_name, 'Total de PRs': total_pull_requests})

    return pd.DataFrame(repo_data)


def analisar_releases(repositorios):
    dados_repo = []
    for repo in repositorios.get('data', {}).get('search', {}).get('nodes', []):
        nome_repo = repo.get('nameWithOwner', '')
        dados_releases = repo.get('releases', {})
        total_releases = dados_releases.get('totalCount', 0)
        dados_repo.append({'Repositório': nome_repo, 'Total de Releases': total_releases})

    return pd.DataFrame(dados_repo)

def analisar_updates(repositorios):
    dados_repo = []
    for repo in repositorios.get('data', {}).get('search', {}).get('nodes', []):
        nome_repo = repo.get('nameWithOwner', '')
        atualizado_em = repo.get('updatedAt', '')
        data_ultimo_update = datetime.strptime(atualizado_em, '%Y-%m-%dT%H:%M:%SZ') if atualizado_em else None
        dias_desde_ultimo_update = (datetime.now() - data_ultimo_update).days if data_ultimo_update else None
        dados_repo.append({'Repositório': nome_repo, 'Data do Último Update': atualizado_em, 'Dias desde o Último Update': dias_desde_ultimo_update})

    return pd.DataFrame(dados_repo)

def analisar_linguagens(repositorios):
    dados_repo = []
    for repo in repositorios.get('data', {}).get('search', {}).get('nodes', []):
        nome_repo = repo.get('nameWithOwner', '')
        linguagem_principal = repo.get('primaryLanguage', {}).get('name') if repo.get('primaryLanguage') else ''
        dados_repo.append({'Repositório': nome_repo, 'Linguagem Mais Comum': linguagem_principal})

    return pd.DataFrame(dados_repo)

def analisar_issues_fechadas(repositorios):
    dados_repo = []
    for repo in repositorios.get('data', {}).get('search', {}).get('nodes', []):
        nome_repo = repo.get('nameWithOwner', '')
        total_issues = repo.get('issues', {}).get('totalCount', 0)
        total_issues_fechadas = repo.get('closedIssues', {}).get('totalCount', 0)

        percentual_issues_fechadas = (total_issues_fechadas / total_issues) * 100 if total_issues > 0 else 0
        dados_repo.append({'Repositório': nome_repo, 'Percentual de Issues Fechadas': percentual_issues_fechadas})

    return pd.DataFrame(dados_repo)

## Variáveis

In [5]:
data = {
    'query': query_geral
}

repositorios_geral = doPost(data)
contribuicoes_analisadas = analisar_contribuicoes(repositorios_geral)
releases_analisadas = analisar_releases(repositorios_geral)
updates_analisados = analisar_updates(repositorios_geral)
linguagens_analisadas = analisar_linguagens(repositorios_geral)
issues_fechadas_analisadas = analisar_issues_fechadas(repositorios_geral)

# Análises

## RQ 01: Sistemas populares são maduros/antigos?
### Métrica: idade do repositório (calculado a partir da data de sua criação)

In [6]:
query_1 = """
query questao_1 {{
    search(query:"stars:>{0}", type:REPOSITORY, first:{0}){{
        nodes {{
            ... on Repository {{
                name
                createdAt
            }}
        }}
    }}
}}
""".format(NUMBER_OF_REPOSITORIES)


In [7]:
repositories = []
data = {
    'query': query_1
}
response_json = doPost(data)
repositories = response_json['data']['search']['nodes']

In [8]:
data_frame = pd.DataFrame(repositories)
created_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ') for date in data_frame['createdAt']]
age_years = [round((TODAY - date).days / 365, 2) for date in created_dates]

data_frame['Age (Years)'] = age_years
mean_age = data_frame['Age (Years)'].mean()
data_frame.head(NUMBER_OF_REPOSITORIES)

# x=10
# if x > 5:
#   print('if')
# elif (x/2 == 0 and x< 5):
#     print('else')

,name,createdAt,Age (Years)
0,freeCodeCamp,2014-12-24T17:49:19Z,9.18
1,free-programming-books,2013-10-11T06:50:37Z,10.38
2,awesome,2014-07-11T13:42:37Z,9.63
3,public-apis,2016-03-20T23:49:42Z,7.94
4,coding-interview-university,2016-06-06T02:34:12Z,7.73
5,996.ICU,2019-03-26T07:31:14Z,4.93
6,developer-roadmap,2017-03-15T13:45:52Z,6.95
7,build-your-own-x,2018-05-09T12:03:18Z,5.81
8,system-design-primer,2017-02-26T16:15:28Z,7.00
9,react,2013-05-24T16:15:54Z,10.76


## RQ 02: Sistemas populares recebem muita contribuição externa?
### Métrica: total de pull requests aceitas

In [9]:
# query_2 = """
# query questao_2 {
#   search(query: "stars:>{0}", type: REPOSITORY, first: {0}) {
#     nodes {
#       ... on Repository {
#         nameWithOwner
#         pullRequests(states:MERGED){
#           totalCount
#         }
#       }
#     }
#   }
# }
# """.format(NUMBER_OF_REPOSITORIES)

In [10]:
contribuicoes_analisadas.head(NUMBER_OF_REPOSITORIES)

,Repositório,Total de PRs
0,freeCodeCamp/freeCodeCamp,21765
1,EbookFoundation/free-programming-books,6428
2,sindresorhus/awesome,641
3,public-apis/public-apis,1871
4,jwasham/coding-interview-university,395
5,996icu/996.ICU,1066
6,kamranahmedse/developer-roadmap,1783
7,codecrafters-io/build-your-own-x,132
8,donnemartin/system-design-primer,196
9,facebook/react,9702


## RQ 03: Sistemas populares lançam releases com frequência?
### Métrica: total de releases

In [11]:
# query_3 = """
# query questao_3 {
#   search(query: "stars:>{}", type: REPOSITORY, first: {0}) {
#     nodes {
#       ... on Repository {
#         nameWithOwner
#         releases{
#           totalCount
#         }
#       }
#     }
#   }
# }
# """.format(NUMBER_OF_REPOSITORIES)

In [12]:
releases_analisadas.head(NUMBER_OF_REPOSITORIES)

,Repositório,Total de Releases
0,freeCodeCamp/freeCodeCamp,0
1,EbookFoundation/free-programming-books,0
2,sindresorhus/awesome,0
3,public-apis/public-apis,0
4,jwasham/coding-interview-university,0
5,996icu/996.ICU,0
6,kamranahmedse/developer-roadmap,1
7,codecrafters-io/build-your-own-x,0
8,donnemartin/system-design-primer,0
9,facebook/react,99


## RQ 04: Sistemas populares são atualizados com frequência?
### Métrica: tempo até a última atualização (calculado a partir da data de última atualização)

In [13]:
# query_4 = """
# query questao_4 {
#   search(query: "stars:> {0}", type: REPOSITORY, first: {0}) {
#     nodes {
#       ... on Repository {
#         nameWithOwner
#         updatedAt
#       }
#     }
#   }
# }
# """.format(NUMBER_OF_REPOSITORIES)

In [14]:
updates_analisados.head(NUMBER_OF_REPOSITORIES)

,Repositório,Data do Último Update,Dias desde o Último Update
0,freeCodeCamp/freeCodeCamp,2024-02-26T13:03:50Z,0
1,EbookFoundation/free-programming-books,2024-02-26T13:04:03Z,0
2,sindresorhus/awesome,2024-02-26T13:04:03Z,0
3,public-apis/public-apis,2024-02-26T13:03:28Z,0
4,jwasham/coding-interview-university,2024-02-26T12:59:26Z,0
5,996icu/996.ICU,2024-02-26T12:09:39Z,0
6,kamranahmedse/developer-roadmap,2024-02-26T13:03:53Z,0
7,codecrafters-io/build-your-own-x,2024-02-26T12:59:16Z,0
8,donnemartin/system-design-primer,2024-02-26T12:49:47Z,0
9,facebook/react,2024-02-26T12:54:47Z,0


## RQ 05: Sistemas populares são escritos nas <a href='https://octoverse.github.com/'>linguagens mais populares</a>?

### Métrica: linguagem primária de cada um desses repositórios

In [15]:
# query_5 = """
# query questao_5 {
#   search(query: "stars:>{0}", type: REPOSITORY, first:{0}) {
#     nodes {
#       ... on Repository {
#         nameWithOwner
#         primaryLanguage {
#           name
#         }
#       }
#     }
#   }
# }
# """.format(NUMBER_OF_REPOSITORIES)

In [16]:
linguagens_analisadas.head(NUMBER_OF_REPOSITORIES)

,Repositório,Linguagem Mais Comum
0,freeCodeCamp/freeCodeCamp,TypeScript
1,EbookFoundation/free-programming-books,
2,sindresorhus/awesome,
3,public-apis/public-apis,Python
4,jwasham/coding-interview-university,
5,996icu/996.ICU,
6,kamranahmedse/developer-roadmap,TypeScript
7,codecrafters-io/build-your-own-x,
8,donnemartin/system-design-primer,Python
9,facebook/react,JavaScript


## RQ 06: Sistemas populares possuem um alto percentual de issues fechadas?
### Métrica: Métrica: razão entre número de issues fechadas pelo total de issues)

In [17]:
# query_6 = """
# query questao_6 {
#   search(query: "stars:>{0}", type: REPOSITORY, first: {0}) {
#     nodes {
#       ... on Repository {
#         nameWithOwner
#         issues{
#           totalCount
#         }
#         closedIssues: issues(states: CLOSED) {
#           totalCount
#         }
#       }
#     }
#   }
# }
# """.format(NUMBER_OF_REPOSITORIES)

In [18]:
issues_fechadas_analisadas.head(NUMBER_OF_REPOSITORIES)

,Repositório,Percentual de Issues Fechadas
0,freeCodeCamp/freeCodeCamp,98.471121
1,EbookFoundation/free-programming-books,97.832234
2,sindresorhus/awesome,93.413174
3,public-apis/public-apis,99.094203
4,jwasham/coding-interview-university,89.104116
5,996icu/996.ICU,0.000000
6,kamranahmedse/developer-roadmap,59.764969
7,codecrafters-io/build-your-own-x,70.764120
8,donnemartin/system-design-primer,28.880866
9,facebook/react,90.818976
